## Data Cleaning & Feature Engineering liliya.chvileva@gmail.com
lowercases for column names
drop: id, name, state: suspended + goal == 0,
features: create duration from launch and end data (datatype conversion)


In [1]:
# Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Load the dataset
df = pd.read_csv('data/kickstarter_projects.csv')


# Lowercases for column names


In [3]:
# Convert column names to lowercase
df.columns = df.columns.str.lower()

In [4]:
# Display the first few rows of the dataframe to understand its structure
print(df.head())

           id                                               name   
0  1860890148  Grace Jones Does Not Give A F$#% T-Shirt (limi...  \
1   709707365                     CRYSTAL ANTLERS UNTITLED MOVIE   
2  1703704063                                drawing for dollars   
3      727286                       Offline Wikipedia iPhone app   
4  1622952265                                         Pantshirts   

       category   subcategory        country             launched    deadline   
0       Fashion       Fashion  United States  2009-04-21 21:02:48  2009-05-31  \
1  Film & Video        Shorts  United States  2009-04-23 00:07:53  2009-07-20   
2           Art  Illustration  United States  2009-04-24 21:52:03  2009-05-03   
3    Technology      Software  United States  2009-04-25 17:36:21  2009-07-14   
4       Fashion       Fashion  United States  2009-04-27 14:10:39  2009-05-26   

    goal  pledged  backers       state  
0   1000      625       30      Failed  
1  80000       22     

# New DataFrame for droped: id, name, state: suspended + goal == 0 and creating a new feauture "duration" of each project

In [13]:


# Drop variables 'id', 'name', and rows where 'state' is 'suspended' or 'goal' is 0
dropped_df = df.drop(columns=['id', 'name']).loc[(df['state'] != 'suspended') & (df['goal'] != 0)]

# Convert 'launched' and 'deadline' columns to datetime
dropped_df['launched'] = pd.to_datetime(dropped_df['launched'])
dropped_df['deadline'] = pd.to_datetime(dropped_df['deadline'])

# Calculate the duration in days
dropped_df['duration'] = (dropped_df['deadline'] - dropped_df['launched']).dt.days

# Save the modified DataFrame to a CSV file in the 'data' folder
dropped_df.to_csv('data/modified_kickstarter_projects.csv', index=False)



In [14]:
print(dropped_df.head())

       category   subcategory        country            launched   deadline   
0       Fashion       Fashion  United States 2009-04-21 21:02:48 2009-05-31  \
1  Film & Video        Shorts  United States 2009-04-23 00:07:53 2009-07-20   
2           Art  Illustration  United States 2009-04-24 21:52:03 2009-05-03   
3    Technology      Software  United States 2009-04-25 17:36:21 2009-07-14   
4       Fashion       Fashion  United States 2009-04-27 14:10:39 2009-05-26   

    goal  pledged  backers       state  duration  
0   1000      625       30      Failed        39  
1  80000       22        3      Failed        87  
2     20       35        3  Successful         8  
3     99      145       25  Successful        79  
4   1900      387       10      Failed        28  


# Trying label encoding

In [18]:
# Trying label encoding
columns_to_encode = df[['category', 'subcategory', 'country']]
# Initialize LabelEncoder
label_encoder = LabelEncoder()
# Apply LabelEncoder to each categorical column
for col in dropped_df:
    df[col + '_encoded'] = label_encoder.fit_transform(df[col])

In [6]:
# Create a DataFrame to store unique original values and their corresponding encoded values for each column
encoded_values_dfs = {}
# Iterate through the columns to be encoded
for col in columns_to_encode:
    # Get unique values of the original column and their corresponding encoded values
    unique_original_values = df[col].unique()
    encoded_values = label_encoder.fit_transform(unique_original_values)
    # Create a DataFrame with original and encoded values
    encoded_values_df = pd.DataFrame({
        f'{col}_Original': unique_original_values,
        f'{col}_Encoded': encoded_values
    })
    # Sort the DataFrame by the encoded values (for 'Category' column)
    if col == 'Category':
        encoded_values_df = encoded_values_df.sort_values(by=f'{col}_Encoded', ascending=True)
    # Store the DataFrame in the dictionary
    encoded_values_dfs[col] = encoded_values_df
# Display the DataFrames
for col, encoded_values_df in encoded_values_dfs.items():
    print(f'\n{col}:\n')
    print(encoded_values_df)


category:

   category_Original  category_Encoded
0            Fashion                 5
1       Film & Video                 6
2                Art                 0
3         Technology                13
4         Journalism                 9
5         Publishing                12
6            Theater                14
7              Music                10
8        Photography                11
9              Games                 8
10            Design                 4
11              Food                 7
12            Crafts                 2
13            Comics                 1
14             Dance                 3

subcategory:

    subcategory_Original  subcategory_Encoded
0                Fashion                   52
1                 Shorts                  129
2           Illustration                   70
3               Software                  131
4             Journalism                   77
..                   ...                  ...
154        Playing Cards   

In [7]:
#save new table

encoded_values_df.to_csv('data/encoded_kickstarter_projects.csv', index=False)